# Учебный Проект → «Золото» ✨

Подготовьте прототип модели машинного обучения для «Цифры». Компания разрабатывает решения для эффективной работы промышленных предприятий.  
Модель должна предсказать коэффициент восстановления золота из золотосодержащей руды. В вашем распоряжении данные с параметрами добычи и очистки.  
Модель поможет оптимизировать производство, чтобы не запускать предприятие с убыточными характеристиками.  
Вам нужно:

- Подготовить данные;
- Провести исследовательский анализ данных;
- Построить и обучить модель;

Метрика качества  
Для решения задачи введём новую метрику качества — **sMAPE** (англ. Symmetric Mean Absolute Percentage Error, «симметричное среднее абсолютное процентное отклонение»).  
Она похожа на MAE, но выражается не в абсолютных величинах, а в относительных.  
Почему симметричная? Она одинаково учитывает масштаб и целевого признака, и предсказания.

$$\large
sMAPE = \frac{1}{N} * \sum_{i=1}^{N} \frac{|y_i - \hat{y_i}|}{(|y_i| + |\hat{y_i}|) / 2} * 100\%
$$

Нужно спрогнозировать сразу две величины:
  
- эффективность обогащения чернового концентрата `rougher.output.recovery`;
- эффективность обогащения финального концентрата `final.output.recovery`;

$$\large
sMAPE = 25\% * sMAPE_{rougher} + 75\% * sMAPE_{final}
$$

## Описание данных

Данные находятся в трёх файлах:

- `gold_recovery_train_new.csv` — обучающая выборка;
- `gold_recovery_test_new.csv` — тестовая выборка;
- `gold_recovery_full_new.csv` — исходные данные;

### Технологический процесс

`Rougher feed` — исходное сырье  
`Rougher additions` (или reagent additions) — флотационные реагенты: Xanthate, Sulphate, Depressant  
- `Xanthate` **— ксантогенат (промотер, или активатор флотации);  
- `Sulphate` — сульфат (на данном производстве сульфид натрия);  
- `Depressant` — депрессант (силикат натрия).  

`Rougher process` (англ. «грубый процесс») — флотация  
`Rougher tails` — отвальные хвосты  
`Float banks` — флотационная установка  
`Cleaner process` — очистка  
`Rougher Au` — черновой концентрат золота  
`Final Au` — финальный концентрат золота  

<hr>

# Загружаем данные

Импортируем библиотеки 🎒

In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import RandomState


from matplotlib import rcParams

from math import sqrt

from statistics import mean
from statistics import stdev

from scipy.stats import t
from scipy.stats import bootstrap

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
from sklearn.utils import shuffle
from sklearn.pipeline import make_pipeline

Настроим вид графиков по+красоте ✨

In [2]:
%config InlineBackend.figure_formats = ['svg']

In [3]:
# style MATPLOTLIBRC
custom_params = {
                'figure.figsize': (10, 6),
                'figure.facecolor': '#232425',
                'figure.dpi': 240,

                'legend.frameon': False,
                'legend.borderpad': 1.4,
                'legend.labelspacing': 0.7,
                'legend.handlelength': 0.7,
                'legend.handleheight': 0.7,

                'axes.facecolor': '#232425',
                'axes.labelcolor': '#EEEEEE',
                'axes.labelpad': 17,
                'axes.spines.left': False,
                'axes.spines.bottom': False,
                'axes.spines.right': False,
                'axes.spines.top': False,
                'axes.grid': False,

                'contour.linewidth': 0.0,

                'xtick.color': '#AAAAAA',
                'ytick.color': '#AAAAAA',
                'xtick.bottom': True,
                'xtick.top': False,
                'ytick.left': True,
                'ytick.right': False,
    
                "lines.color": '#EEEEEE',

                'text.color': '#EEEEEE',
    
                'font.family': 'sans-serif',
                # 'font.sans-serif': [
                #     'Helvetica',
                #     'Verdana',
                #     'Tahoma',
                #     'Trebuchet MS',
                #     'Arial',
                #     'Chevin'
                #     ]
                }

# rcParams.update(custom_params)

Константы.

In [4]:
random_seed = 108108108
random_np = RandomState(128) 
dpi_k = custom_params['figure.dpi'] / rcParams['figure.dpi']
px = 1/custom_params['figure.dpi']

Функции.

In [5]:
def to_px(size_px):
    px_density = int(
        round(
            size_px * px * dpi_k
        )
    )
    return px_density

In [6]:
def baisic_df_info(data_df, title='Basic Info'):
    print(title, end='\n\n')
    print('Дубликатов:',
             len(data_df.loc[data_df.duplicated()].index),
          end='\n\n'
     )
    
    display(
        data_df.info(),
        data_df.sample(5),
        data_df.describe(),
    )

<hr>

$$\large
recovery = \frac {C * (F - T) }{F * (C -T)} * 100\%
$$

где:

- C — `rougher.output.concentrate_au` доля золота в концентрате после флотации/очистки;
- F — `rougher.input.feed_au` доля золота в сырье/концентрате до флотации/очистки;
- T — `rougher.output.tail_au` доля золота в отвальных хвостах после флотации/очистки;

In [51]:
def calc_recovery(concentrate_au, feed_au, tail_au):
    c = concentrate_au
    f = feed_au
    t = tail_au
    
    recovery = ((c * (f - t)) / (f * (c - t))) * 100
    
    return recovery

## 1. Подготовим данные

### 1.1. Загрузим файлы и изучим их.

> Данные индексируются датой и временем получения информации (признак date)

In [7]:
try:
    gold_recovery_train = pd.read_csv(
        './datasets/gold_recovery_train_new.csv',
        index_col='date',
    )
    gold_recovery_test = pd.read_csv(
        './datasets/gold_recovery_test_new.csv',
        index_col='date',
    )
    gold_recovery_full = pd.read_csv(
        './datasets/gold_recovery_full_new.csv',
        index_col='date',
    )
    
except FileNotFoundError:
    gold_recovery_train = pd.read_csv(
        'https://code.s3.yandex.net/datasets/gold_recovery_train_new.csv',
        index_col='date',
    )
    gold_recovery_test = pd.read_csv(
        'https://code.s3.yandex.net/datasets/gold_recovery_test_new.csv',
        index_col='date',
    )
    gold_recovery_full = pd.read_csv(
        'https://code.s3.yandex.net/datasets/gold_recovery_full_new.csv',
        index_col='date',
    )
    print('FYI datasets loaded via url')

In [8]:
gold_recovery_train.index = pd.to_datetime(gold_recovery_train.index)
gold_recovery_test.index = pd.to_datetime(gold_recovery_test.index)
gold_recovery_full.index = pd.to_datetime(gold_recovery_full.index)

In [9]:
baisic_df_info(gold_recovery_train, 'Обучающая Выборка')

Обучающая Выборка

Дубликатов: 0

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 14149 entries, 2016-01-15 00:00:00 to 2018-08-18 10:59:59
Data columns (total 86 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   final.output.concentrate_ag                         14148 non-null  float64
 1   final.output.concentrate_pb                         14148 non-null  float64
 2   final.output.concentrate_sol                        13938 non-null  float64
 3   final.output.concentrate_au                         14149 non-null  float64
 4   final.output.recovery                               14149 non-null  float64
 5   final.output.tail_ag                                14149 non-null  float64
 6   final.output.tail_pb                                14049 non-null  float64
 7   final.output.tail_sol                               14144 non-null  float64
 8   final.o

None

,final.output.concentrate_ag,final.output.concentrate_pb,final.output.concentrate_sol,final.output.concentrate_au,final.output.recovery,final.output.tail_ag,final.output.tail_pb,final.output.tail_sol,final.output.tail_au,primary_cleaner.input.sulfate,...,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
date,,,,,,,,,,,,,,,,,,,,,
2018-03-06 06:59:59,6.047899,10.187796,9.028004,43.702739,74.458448,13.733954,3.513998,8.761103,3.224725,142.152367,...,23.042611,-499.448132,14.884886,-500.040188,17.979582,-499.806470,12.007265,-500.084060,11.981150,-500.035243
2016-08-25 08:59:59,4.493924,10.833346,8.505341,45.582788,68.631837,8.581861,2.866737,7.946325,3.312653,134.470214,...,17.982493,-400.259842,9.965597,-400.050154,10.014318,-399.487596,4.974389,-399.758043,25.001851,-499.341467
2018-06-06 18:59:59,5.273264,10.963426,5.955299,42.481690,72.631510,7.522435,1.847169,10.684430,1.835490,65.637147,...,29.985990,-498.992309,21.929303,-499.960703,24.999220,-499.466515,22.999044,-499.891697,24.994454,-499.658430
2017-07-05 20:59:59,4.492270,10.601609,10.480016,45.095888,68.261413,7.595020,2.759236,9.923751,2.477229,151.857222,...,22.053009,-500.856272,15.079991,-381.122889,17.981407,-500.370501,13.026144,-500.271635,15.007860,-500.073753
2016-07-14 21:59:59,6.889989,6.556996,12.351730,45.001447,75.135729,9.484624,1.783621,9.279501,2.114181,82.959563,...,15.035580,-399.300512,4.921690,-399.795267,10.034196,-299.739978,5.051699,-400.778588,25.026022,-451.141311


,final.output.concentrate_ag,final.output.concentrate_pb,final.output.concentrate_sol,final.output.concentrate_au,final.output.recovery,final.output.tail_ag,final.output.tail_pb,final.output.tail_sol,final.output.tail_au,primary_cleaner.input.sulfate,...,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
count,14148.000000,14148.000000,13938.000000,14149.000000,14149.000000,14149.000000,14049.000000,14144.000000,14149.000000,14129.000000,...,14143.000000,14148.000000,14148.000000,14148.000000,14148.000000,14148.000000,14148.000000,14148.000000,14147.000000,14148.000000
mean,5.142034,10.132960,9.202849,44.003792,66.518832,9.607035,2.597298,10.512122,2.918421,133.320659,...,19.985454,-478.696836,15.487065,-460.229416,16.775136,-483.956022,13.064590,-483.966564,19.577539,-506.798480
std,1.369586,1.654930,2.790516,4.905261,10.295402,2.319069,0.971843,3.003617,0.903712,39.431659,...,5.657723,50.736021,5.255655,58.843586,5.831906,37.892788,5.765617,39.207913,5.764417,37.079249
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003112,...,0.000000,-799.709069,0.000000,-799.889113,-0.372054,-797.142475,0.646208,-800.006180,0.195324,-809.398668
25%,4.211620,9.297355,7.484645,43.276111,62.545817,7.997429,1.905973,8.811324,2.368607,107.006651,...,14.990775,-500.628656,11.894558,-500.149000,11.083980,-500.363177,8.994405,-500.105994,14.989304,-500.745104
50%,4.994652,10.297144,8.845462,44.872436,67.432775,9.480270,2.592022,10.514621,2.851025,133.018328,...,20.001789,-499.681450,14.975536,-499.388738,17.932223,-499.702452,11.997547,-499.914556,19.984175,-500.061431
75%,5.859540,11.170603,10.487508,46.166425,72.346428,11.003707,3.241723,11.933009,3.434764,159.825396,...,24.990826,-477.472413,20.059375,-400.039008,21.346550,-487.712108,17.982903,-453.186936,24.991623,-499.536466
max,16.001945,17.031899,18.124851,52.756638,100.000000,19.552149,5.639565,22.317730,8.197408,250.127834,...,30.115735,-245.239184,24.007913,-145.071088,43.709931,-275.073125,27.926001,-157.396071,32.188906,-104.427459


In [10]:
baisic_df_info(gold_recovery_test, 'Тестовая Выборка')

Тестовая Выборка

Дубликатов: 0

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5290 entries, 2016-09-01 00:59:59 to 2017-12-31 23:59:59
Data columns (total 52 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   primary_cleaner.input.sulfate               5286 non-null   float64
 1   primary_cleaner.input.depressant            5285 non-null   float64
 2   primary_cleaner.input.feed_size             5290 non-null   float64
 3   primary_cleaner.input.xanthate              5286 non-null   float64
 4   primary_cleaner.state.floatbank8_a_air      5290 non-null   float64
 5   primary_cleaner.state.floatbank8_a_level    5290 non-null   float64
 6   primary_cleaner.state.floatbank8_b_air      5290 non-null   float64
 7   primary_cleaner.state.floatbank8_b_level    5290 non-null   float64
 8   primary_cleaner.state.floatbank8_c_air      5290 non-null   float64
 9   primary_cleaner.

None

,primary_cleaner.input.sulfate,primary_cleaner.input.depressant,primary_cleaner.input.feed_size,primary_cleaner.input.xanthate,primary_cleaner.state.floatbank8_a_air,primary_cleaner.state.floatbank8_a_level,primary_cleaner.state.floatbank8_b_air,primary_cleaner.state.floatbank8_b_level,primary_cleaner.state.floatbank8_c_air,primary_cleaner.state.floatbank8_c_level,...,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
date,,,,,,,,,,,,,,,,,,,,,
2017-12-10 12:59:59,211.617021,10.490017,8.195000,1.101640,1546.349042,-499.620225,1549.771138,-497.809872,1549.542594,-499.884324,...,20.026320,-499.462603,14.978411,-145.664336,10.921761,-499.168532,7.998631,-500.034426,11.991212,-499.415556
2016-12-15 07:59:59,188.098380,12.017905,6.908334,0.994718,1501.604646,-499.501792,1501.329021,-500.249448,1501.364082,-498.017417,...,17.001373,-499.874250,15.056428,-500.396148,17.773944,-499.969826,11.957138,-500.419854,20.990542,-498.923240
2016-11-09 19:59:59,154.027197,5.048973,7.353334,0.922994,1604.339061,-500.552423,1597.604681,-499.565345,1598.900052,-500.050638,...,16.009283,-500.584137,14.026107,-499.309153,16.026756,-502.420807,13.990113,-500.621473,21.994704,-501.566064
2016-09-07 18:59:59,137.957334,11.062530,8.050000,0.659894,1298.552905,-500.126807,1298.766516,-499.435003,1297.686064,-494.323296,...,7.998738,-499.332131,4.909812,-599.189134,8.042765,-554.349860,2.882059,-599.763041,20.003762,-598.885893
2016-10-12 21:59:59,205.639112,9.054495,8.170000,0.608262,1498.291811,-500.781679,1500.462106,-499.683377,1500.243312,-502.320398,...,12.007678,-501.674473,11.986107,-500.527298,9.960917,-500.717687,8.031101,-498.950468,16.956322,-500.200492


,primary_cleaner.input.sulfate,primary_cleaner.input.depressant,primary_cleaner.input.feed_size,primary_cleaner.input.xanthate,primary_cleaner.state.floatbank8_a_air,primary_cleaner.state.floatbank8_a_level,primary_cleaner.state.floatbank8_b_air,primary_cleaner.state.floatbank8_b_level,primary_cleaner.state.floatbank8_c_air,primary_cleaner.state.floatbank8_c_level,...,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
count,5286.000000,5285.000000,5290.000000,5286.000000,5.290000e+03,5290.000000,5.290000e+03,5290.000000,5.290000e+03,5290.000000,...,5.290000e+03,5290.000000,5.290000e+03,5290.000000,5290.000000,5290.000000,5290.000000,5290.000000,5290.000000,5290.000000
mean,174.839652,8.683596,7.266339,1.383803,1.539494e+03,-497.665883,1.545174e+03,-500.273098,1.527272e+03,-498.330680,...,1.632007e+01,-505.144570,1.373544e+01,-463.349858,12.804186,-501.329122,9.881145,-495.663398,17.304935,-501.793193
std,43.027080,3.072050,0.610219,0.643474,1.167979e+02,19.952431,1.222246e+02,32.968307,1.225380e+02,21.964876,...,3.493583e+00,31.427337,3.430484e+00,86.189107,3.026591,17.951495,2.868205,34.535007,4.536544,39.044215
min,2.566156,0.003839,5.650000,0.004984,5.445860e-32,-795.316337,6.647490e-32,-799.997015,4.033736e-32,-799.960571,...,1.079872e-16,-799.798523,2.489718e-17,-800.836914,0.069227,-797.323986,0.528083,-800.220337,-0.079426,-809.741464
25%,147.121401,6.489555,6.890000,0.907623,1.498936e+03,-500.357298,1.498971e+03,-500.703002,1.473230e+03,-501.018117,...,1.403618e+01,-500.868258,1.202862e+01,-500.323028,10.914838,-500.726841,8.036719,-500.194668,13.997317,-500.690984
50%,177.828489,8.052207,7.250000,1.197610,1.585129e+03,-499.969164,1.595622e+03,-500.028514,1.549595e+03,-500.017711,...,1.700847e+01,-500.115727,1.496486e+01,-499.576513,12.954182,-499.990332,10.004301,-499.990535,16.014935,-500.007126
75%,208.125438,10.027764,7.600000,1.797819,1.602077e+03,-499.568951,1.602324e+03,-499.293257,1.601144e+03,-498.994130,...,1.803862e+01,-499.404224,1.596213e+01,-400.933805,15.097528,-499.283191,11.997467,-499.719913,21.020013,-499.373018
max,265.983123,40.000000,15.500000,4.102454,2.103104e+03,-57.195404,1.813084e+03,-142.527229,1.715054e+03,-150.937035,...,3.005180e+01,-401.565212,3.126971e+01,-6.506986,25.258848,-244.483566,14.086866,-137.740004,26.705889,-123.307487


In [11]:
baisic_df_info(gold_recovery_full, 'Исходные Данные')

Исходные Данные

Дубликатов: 0

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 19439 entries, 2016-01-15 00:00:00 to 2018-08-18 10:59:59
Data columns (total 86 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   final.output.concentrate_ag                         19438 non-null  float64
 1   final.output.concentrate_pb                         19438 non-null  float64
 2   final.output.concentrate_sol                        19228 non-null  float64
 3   final.output.concentrate_au                         19439 non-null  float64
 4   final.output.recovery                               19439 non-null  float64
 5   final.output.tail_ag                                19438 non-null  float64
 6   final.output.tail_pb                                19338 non-null  float64
 7   final.output.tail_sol                               19433 non-null  float64
 8   final.out

None

,final.output.concentrate_ag,final.output.concentrate_pb,final.output.concentrate_sol,final.output.concentrate_au,final.output.recovery,final.output.tail_ag,final.output.tail_pb,final.output.tail_sol,final.output.tail_au,primary_cleaner.input.sulfate,...,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
date,,,,,,,,,,,,,,,,,,,,,
2017-08-09 01:59:59,4.173100,11.874376,7.035053,45.276338,69.444081,7.844479,4.170360,11.790037,2.774370,186.780349,...,18.062028,-500.158317,12.123342,-400.070533,15.945628,-499.868732,11.985255,-500.110502,14.976551,-500.130052
2016-06-08 07:59:59,4.451519,10.378770,7.200758,45.762838,58.047659,8.699252,2.220734,7.631626,3.286378,81.267510,...,8.951855,-549.675334,7.061309,-548.423311,7.949243,-549.934816,6.043796,-550.190407,18.046440,-548.983690
2016-08-12 21:59:59,4.477596,9.007425,11.350100,46.753173,71.560428,10.542143,3.201486,11.295260,4.205234,176.122189,...,19.967526,-404.912573,9.880538,-409.065950,16.282064,-397.236440,5.034202,-400.498470,23.017855,-498.625244
2018-04-25 15:59:59,5.090715,10.086001,8.381159,46.257578,75.423379,11.359790,1.897141,7.983995,2.378571,170.375692,...,27.000162,-499.447075,19.967401,-500.865925,19.947301,-498.248853,15.018667,-500.211089,17.991384,-500.144310
2018-04-29 21:59:59,5.955403,11.452148,8.010769,42.527939,75.962757,9.496956,2.034912,8.264648,2.053974,164.842666,...,29.970844,-498.497610,20.053210,-498.696072,24.981928,-500.731095,22.999071,-500.174585,25.023527,-500.974370


,final.output.concentrate_ag,final.output.concentrate_pb,final.output.concentrate_sol,final.output.concentrate_au,final.output.recovery,final.output.tail_ag,final.output.tail_pb,final.output.tail_sol,final.output.tail_au,primary_cleaner.input.sulfate,...,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
count,19438.000000,19438.000000,19228.000000,19439.000000,19439.000000,19438.000000,19338.000000,19433.000000,19439.000000,19415.000000,...,19433.000000,19438.000000,19438.000000,19438.000000,19438.000000,19438.000000,19438.000000,19438.000000,19437.000000,19438.000000
mean,5.168470,9.978895,9.501224,44.076513,67.050208,9.688589,2.705795,10.583728,3.042467,144.624774,...,18.987674,-485.894516,15.010366,-461.078636,15.694452,-488.684065,12.198224,-487.149827,18.959024,-505.436305
std,1.372348,1.669240,2.787537,5.129784,10.125840,2.328642,0.949077,2.868782,0.922808,44.464071,...,5.411058,47.758570,4.890228,67.405524,5.510974,34.533396,5.333024,38.347312,5.550498,37.689057
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003112,...,0.000000,-799.798523,0.000000,-800.836914,-0.372054,-797.323986,0.528083,-800.220337,-0.079426,-809.741464
25%,4.251240,9.137262,7.722820,43.402215,63.299712,8.062878,2.040119,8.938947,2.461138,114.107029,...,14.975734,-500.704892,11.940294,-500.187742,10.988606,-500.458467,8.971105,-500.129462,14.983037,-500.728067
50%,5.066094,10.102433,9.218961,45.011244,68.172738,9.743623,2.748730,10.622456,2.984909,143.232103,...,18.017481,-499.837793,14.971014,-499.459786,15.000036,-499.802605,11.019433,-499.935317,19.960541,-500.048400
75%,5.895527,11.035769,10.947813,46.275313,72.686642,11.134294,3.333216,12.104271,3.571351,175.075656,...,23.012470,-498.249730,19.034162,-400.118106,18.026190,-498.384187,14.019088,-499.436708,24.003170,-499.495378
max,16.001945,17.031899,19.615720,52.756638,100.000000,19.552149,5.804178,22.317730,8.245022,265.983123,...,30.115735,-245.239184,31.269706,-6.506986,43.709931,-244.483566,27.926001,-137.740004,32.188906,-104.427459


### 1.2. Проверим, что эффективность обогащения рассчитана правильно

####  Вычислим эффективность обогащения на обучающей выборке для признака `gold_recovery_train['rougher.output.recovery']`

$$\large
recovery = \frac {C * (F - T) }{F * (C -T)} * 100\%
$$

где:

- C — `rougher.output.concentrate_au` доля золота в концентрате после флотации/очистки;
- F — `rougher.input.feed_au` доля золота в сырье/концентрате до флотации/очистки;
- T — `rougher.output.tail_au` доля золота в отвальных хвостах после флотации/очистки;

In [50]:
print(
    'Строк в тренировочной выборке:',
    len(gold_recovery_train.index)
)

Строк в тренировочной выборке: 14149


In [ ]:
display(
    len(gold_recovery_train.loc[
        gold_recovery_train['rougher.output.recovery'].notna()
    ].index),
    len(gold_recovery_train.loc[
        gold_recovery_train['rougher.output.concentrate_au'].notna()
    ].index),
    len(gold_recovery_train.loc[
        gold_recovery_train['rougher.input.feed_au'].notna()
    ].index),
    len(gold_recovery_train.loc[
        gold_recovery_train['rougher.output.tail_au'].notna()
    ].index),
)

14149

14149

14149

14149

In [32]:
display(
    gold_recovery_train['rougher.output.recovery'].notna().sum(),
    gold_recovery_train['rougher.output.concentrate_au'].notna().sum(),
    gold_recovery_train['rougher.input.feed_au'].notna().sum(),
    gold_recovery_train['rougher.output.tail_au'].notna().sum(),
)

14149

14149

14149

14149

Данные для вычислений на месте)

$$\large
recovery = \frac {C * (F - T) }{F * (C -T)} * 100\%
$$

In [53]:
gold_recovery_train['rougher.calculation.recovery'] = gold_recovery_train.loc[:, 
        ['rougher.output.concentrate_au',
        'rougher.input.feed_au',
        'rougher.output.tail_au',]
    ].apply(lambda to_calc:
            calc_recovery(
                to_calc['rougher.output.concentrate_au'],
                to_calc['rougher.input.feed_au'],
                to_calc['rougher.output.tail_au']
            ),
            axis=1
    )

In [58]:
display(
    gold_recovery_train.loc[: ,
        ['rougher.calculation.recovery', 'rougher.output.recovery']
    ].sample(5)
)

,rougher.calculation.recovery,rougher.output.recovery
date,,
2016-04-13 02:00:00,80.621123,80.621123
2017-08-28 16:59:59,88.077188,88.077188
2018-06-01 12:59:59,91.852340,91.852340
2016-07-31 19:59:59,77.250592,77.250592
2018-08-04 04:59:59,93.109028,93.109028


####  Найдём MAE между расчётами и значением признака

$$\large
MAE = \frac{1}{N} * \sum_{i=1}^{N} |y_i - x_i|
$$

In [60]:
print(
    'MAE между расчётами и значением признака rougher.output.recovery',
    mean_absolute_error(
        gold_recovery_train['rougher.output.recovery'],
        gold_recovery_train['rougher.calculation.recovery'],
    ),
    sep=':\n'
)

MAE между расчётами и значением признака rougher.output.recovery:
9.73512347450521e-15


In [49]:
display(
    gold_recovery_train.loc[
        gold_recovery_train['rougher.output.recovery'] != gold_recovery_train['rougher.calculation.recovery'],
        ['rougher.output.recovery', 'rougher.calculation.recovery']
    ][:5].apply(
        lambda to_check: print(
            to_check['rougher.output.recovery'],
            to_check['rougher.calculation.recovery']
        ),
        axis=1
    )
)

86.84326050586624 86.84326050586625
86.84230825746624 86.84230825746626
88.15691183260715 88.15691183260716
88.16806533451772 88.1680653345177
87.03586230135834 87.03586230135835


date
2016-01-15 01:00:00    None
2016-01-15 02:00:00    None
2016-01-15 05:00:00    None
2016-01-15 06:00:00    None
2016-01-15 08:00:00    None
dtype: object

~~На всякий случай глянем в `gold_recovery_full`~~

####  Выводы

MAE, конечно, довольно мало, но вот вопрос как это $MAE \neq 0$ !?  
Похоже, дело в `python` и его бинарной особенности в работе с `float`..🐍

### 1.3. Проанализируем признаки, недоступные в тестовой выборке.

> Некоторые параметры недоступны, потому что замеряются и/или рассчитываются значительно позже. Из-за этого в тестовой выборке отсутствуют некоторые признаки, которые могут быть в обучающей. Также в тестовом наборе нет целевых признаков.  
Исходный датасет содержит обучающую и тестовую выборки со всеми признаками.  

> В вашем распоряжении сырые данные: их просто выгрузили из хранилища. Прежде чем приступить к построению модели, проверьте по нашей инструкции их на корректность.

####  Что это за параметры?

Посмотрим какие колонки содержат `NaN`:

In [32]:
test_col_na = gold_recovery_test.columns[
        gold_recovery_test.isna().any()
    ].tolist()
display(
    gold_recovery_test.loc[: , test_col_na].info()
)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5290 entries, 2016-09-01 00:59:59 to 2017-12-31 23:59:59
Data columns (total 12 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   primary_cleaner.input.sulfate             5286 non-null   float64
 1   primary_cleaner.input.depressant          5285 non-null   float64
 2   primary_cleaner.input.xanthate            5286 non-null   float64
 3   rougher.input.feed_rate                   5287 non-null   float64
 4   rougher.input.feed_size                   5289 non-null   float64
 5   rougher.input.feed_sol                    5269 non-null   float64
 6   rougher.input.floatbank10_sulfate         5285 non-null   float64
 7   rougher.input.floatbank11_sulfate         5282 non-null   float64
 8   rougher.input.floatbank11_xanthate        5265 non-null   float64
 9   secondary_cleaner.state.floatbank2_a_air  5287 non-null   float64
 10  

None

#### К какому типу относятся?

Если вопрос о типе данных, то `float64`  
Если вопрос о `[тип_параметра]`, то:
  
- **input** — параметры сырья;
- **state** — параметры, характеризующие текущее состояние этапа;

In [48]:
not_in_test = list(
        set(gold_recovery_train.columns)
        - set(gold_recovery_test.columns)
    )

PHASE_ORDER = {
    'rougher': 'a',
    'primary_cleaner': 'b',
    'secondary_cleaner': 'c',
    'final': 'd',
}

TYPE_ORDER = {
    'input': 'a',
    'output': 'b',
    'state': 'c',
    'calculation': 'd',
}

# mylist.sort(key=lambda val: SORT_ORDER[val[1]])

display(
    len(not_in_test),
    sorted(
        not_in_test,
        key=lambda val: PHASE_ORDER[val.split('.')[0]] + TYPE_ORDER[val.split('.')[1]]
    )
)

34

['rougher.output.concentrate_pb',
 'rougher.output.tail_pb',
 'rougher.output.recovery',
 'rougher.output.tail_ag',
 'rougher.output.tail_sol',
 'rougher.output.concentrate_sol',
 'rougher.output.tail_au',
 'rougher.output.concentrate_au',
 'rougher.output.concentrate_ag',
 'rougher.calculation.floatbank11_sulfate_to_au_feed',
 'rougher.calculation.sulfate_to_au_concentrate',
 'rougher.calculation.floatbank10_sulfate_to_au_feed',
 'rougher.calculation.au_pb_ratio',
 'primary_cleaner.output.tail_pb',
 'primary_cleaner.output.concentrate_sol',
 'primary_cleaner.output.concentrate_pb',
 'primary_cleaner.output.concentrate_au',
 'primary_cleaner.output.concentrate_ag',
 'primary_cleaner.output.tail_sol',
 'primary_cleaner.output.tail_ag',
 'primary_cleaner.output.tail_au',
 'secondary_cleaner.output.tail_pb',
 'secondary_cleaner.output.tail_sol',
 'secondary_cleaner.output.tail_au',
 'secondary_cleaner.output.tail_ag',
 'final.output.concentrate_pb',
 'final.output.concentrate_sol',
 'fina

In [38]:
display(
    gold_recovery_full.loc[: , not_in_test].sample(17)
)

,primary_cleaner.output.tail_pb,secondary_cleaner.output.tail_pb,rougher.output.concentrate_pb,final.output.concentrate_pb,rougher.output.tail_pb,rougher.output.recovery,final.output.concentrate_sol,rougher.output.tail_ag,final.output.tail_ag,primary_cleaner.output.concentrate_sol,...,primary_cleaner.output.tail_ag,rougher.calculation.floatbank10_sulfate_to_au_feed,final.output.tail_au,final.output.concentrate_au,secondary_cleaner.output.tail_au,final.output.tail_pb,final.output.tail_sol,primary_cleaner.output.tail_au,rougher.calculation.au_pb_ratio,secondary_cleaner.output.tail_ag
date,,,,,,,,,,,,,,,,,,,,,
2017-06-08 09:59:59,3.213112,9.422876,8.543355,10.249978,0.672537,79.738688,7.868413,6.360942,8.745753,9.577914,...,14.994405,6457.978287,3.431724,45.029618,9.284144,3.477239,9.302840,3.811789,2.003032,15.151397
2018-07-07 23:59:59,3.995507,11.517235,9.132172,11.794190,0.161126,85.196095,6.461700,4.446307,6.032508,7.543806,...,17.391734,2778.082945,0.929239,45.785115,2.538883,1.633096,2.752751,1.835326,1.406142,17.696667
2017-04-19 01:59:59,1.001899,2.741768,7.314317,11.224532,0.717441,88.477625,12.178571,5.942185,8.442453,6.397821,...,10.831200,3284.525709,2.331353,44.215998,3.600147,2.715915,13.534259,2.581741,2.095103,10.279825
2017-09-02 06:59:59,1.297189,3.681453,5.528345,5.278167,0.508341,64.120213,4.819082,3.679518,3.124735,3.937464,...,4.751073,3231.604707,1.917090,21.566522,0.943460,2.363919,7.378626,1.547862,1.812917,6.576014
2017-10-16 12:59:59,3.092496,7.481333,8.497636,11.524012,0.765467,87.850636,8.986270,4.102178,8.030128,3.150483,...,14.606559,3427.703703,2.975066,41.365847,5.185835,3.025014,12.297910,4.762318,2.274437,17.032830
2016-06-28 02:59:59,1.013824,0.986197,5.472338,6.667300,0.589437,73.243281,8.662737,6.073654,7.391568,9.142767,...,12.715050,3665.606083,1.899280,49.165009,3.443982,0.873376,13.630273,3.773984,3.478098,12.517044
2018-02-22 10:59:59,4.895714,7.880678,7.885734,10.654272,0.530107,92.054853,8.170367,4.198964,9.584253,11.256031,...,20.770542,2519.033476,2.065868,44.642991,3.721740,1.759793,10.127897,3.055455,2.181547,18.737701
2016-09-03 01:59:59,2.933827,6.167881,9.452184,10.574645,1.212769,66.660027,7.679684,6.828774,7.674371,5.604519,...,13.873487,2386.440035,4.316195,47.185937,6.062614,3.328298,10.385048,5.646028,1.866808,11.772497
2018-07-09 15:59:59,6.526677,12.569924,12.420549,11.733145,0.620917,92.137766,6.818121,4.376589,10.438830,10.757610,...,22.749929,2783.417266,1.953170,46.468276,3.103311,4.167343,7.948003,2.712362,1.123066,16.820871





1.3. Проанализируйте признаки, недоступные в тестовой выборке. Что это за параметры? К какому типу относятся?
1.4. Проведите предобработку данных.
2. Проанализируйте данные
2.1. Посмотрите, как меняется концентрация металлов (Au, Ag, Pb) на различных этапах очистки. Опишите выводы.
2.2. Сравните распределения размеров гранул сырья на обучающей и тестовой выборках. Если распределения сильно отличаются друг от друга, оценка модели будет неправильной.
2.3. Исследуйте суммарную концентрацию всех веществ на разных стадиях: в сырье, в черновом и финальном концентратах. 